In [1]:
# pip install zipcodes
import pandas as pd
import numpy as np
import json
import zipcodes
# file = pd.read_csv('loan.csv')
# df = pd.DataFrame(file)

In [2]:
file = pd.read_csv('loan.csv', usecols = [ 'addr_state','zip_code', 'loan_amnt','annual_inc'])
loan_amnt = pd.DataFrame(file)

In [3]:
loan_amnt_zip = loan_amnt.groupby(['addr_state','zip_code']).agg({'loan_amnt': np.mean, 'annual_inc': np.mean}).reset_index()
loan_amnt_zip.head()

,addr_state,zip_code,loan_amnt,annual_inc
0,AK,009xx,21625.0,75000.0
1,AK,025xx,30000.0,100000.0
2,AK,090xx,35000.0,109000.0
3,AK,112xx,8000.0,68000.0
4,AK,300xx,5400.0,55000.0


In [4]:

def convertzip_county(zip):
    zip = zip[0:3]
    county = zipcodes.similar_to(zip)
    if county:
        county = county[0]['state']
#         county = county.rsplit(' ', 1)[0]
    else: county = ""
    return county


loan_amnt_zip['county'] = loan_amnt_zip['zip_code'].apply(convertzip_county)
loan_amnt_zip.head()

,addr_state,zip_code,loan_amnt,annual_inc,county
0,AK,009xx,21625.0,75000.0,PR
1,AK,025xx,30000.0,100000.0,MA
2,AK,090xx,35000.0,109000.0,AE
3,AK,112xx,8000.0,68000.0,NY
4,AK,300xx,5400.0,55000.0,GA


In [5]:
loan_amnt_zip_new = loan_amnt_zip.loc[loan_amnt_zip.addr_state == loan_amnt_zip.county]

In [6]:
def convertzip_county(zip):
    zip = zip[0:3]
    county = zipcodes.similar_to(zip)
    if county:
        county = county[0]['county']
        county = county.rsplit(' ', 1)[0]
    else: county = ""
    return county


loan_amnt_zip_new['county'] = loan_amnt_zip_new['zip_code'].apply(convertzip_county)
loan_amnt_zip_new.head()

C:\Users\ivyll\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


,addr_state,zip_code,loan_amnt,annual_inc,county
34,AK,995xx,17210.010020,80652.832982,Anchorage
35,AK,996xx,18103.088723,83295.471302,Yukon-Koyukuk Census
36,AK,997xx,17266.480892,77775.333875,Fairbanks North Star
37,AK,998xx,16078.203518,72974.729271,Juneau City and
38,AK,999xx,16130.427632,68294.797303,Ketchikan Gateway


In [7]:
loan_amnt_zip_new.head()

,addr_state,zip_code,loan_amnt,annual_inc,county
34,AK,995xx,17210.010020,80652.832982,Anchorage
35,AK,996xx,18103.088723,83295.471302,Yukon-Koyukuk Census
36,AK,997xx,17266.480892,77775.333875,Fairbanks North Star
37,AK,998xx,16078.203518,72974.729271,Juneau City and
38,AK,999xx,16130.427632,68294.797303,Ketchikan Gateway


In [8]:
loan_amnt_state = loan_amnt.groupby(['addr_state']).agg({'loan_amnt': np.mean, 'annual_inc': np.mean}).reset_index()
loan_amnt_state.head()

,addr_state,loan_amnt,annual_inc
0,AK,17285.394762,79823.042942
1,AL,14686.235523,70854.753011
2,AR,14094.558979,67588.154209
3,AZ,14522.307399,74186.970789
4,CA,15287.680784,83847.981089


In [9]:
state_codes = {
    'WA': '53', 'DE': '10', 'DC': '11', 'WI': '55', 'WV': '54', 'HI': '15',
    'FL': '12', 'WY': '56', 'PR': '72', 'NJ': '34', 'NM': '35', 'TX': '48',
    'LA': '22', 'NC': '37', 'ND': '38', 'NE': '31', 'TN': '47', 'NY': '36',
    'PA': '42', 'AK': '02', 'NV': '32', 'NH': '33', 'VA': '51', 'CO': '08',
    'CA': '06', 'AL': '01', 'AR': '05', 'VT': '50', 'IL': '17', 'GA': '13',
    'IN': '18', 'IA': '19', 'MA': '25', 'AZ': '04', 'ID': '16', 'CT': '09',
    'ME': '23', 'MD': '24', 'OK': '40', 'OH': '39', 'UT': '49', 'MO': '29',
    'MN': '27', 'MI': '26', 'RI': '44', 'KS': '20', 'MT': '30', 'MS': '28',
    'SC': '45', 'KY': '21', 'OR': '41', 'SD': '46'
}

loan_amnt_zip_new['state'] = loan_amnt_zip_new['addr_state'].apply(lambda x: state_codes[x])
loan_amnt_state['state'] = loan_amnt_state['addr_state'].apply(lambda x: state_codes[x])

C:\Users\ivyll\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [10]:
loan_amnt_state=loan_amnt_state.rename(columns = {'addr_state':'id'})


loan_amnt_zip_new=loan_amnt_zip_new.rename(columns = {'addr_state':'id'})


loan_amnt_state = json.loads(loan_amnt_state.to_json(orient='records'))
loan_amnt_zip_new = json.loads(loan_amnt_zip_new.to_json(orient='records'))

In [11]:
loan_amnt_state

[{'id': 'AK',
  'loan_amnt': 17285.3947619958,
  'annual_inc': 79823.0429420761,
  'state': '02'},
 {'id': 'AL',
  'loan_amnt': 14686.2355226506,
  'annual_inc': 70854.7530105557,
  'state': '01'},
 {'id': 'AR',
  'loan_amnt': 14094.5589785639,
  'annual_inc': 67588.1542093241,
  'state': '05'},
 {'id': 'AZ',
  'loan_amnt': 14522.307399074,
  'annual_inc': 74186.970789185,
  'state': '04'},
 {'id': 'CA',
  'loan_amnt': 15287.6807838923,
  'annual_inc': 83847.9810885343,
  'state': '06'},
 {'id': 'CO',
  'loan_amnt': 15103.3398709088,
  'annual_inc': 76948.3125060706,
  'state': '08'},
 {'id': 'CT',
  'loan_amnt': 15301.2980299008,
  'annual_inc': 86531.0308553864,
  'state': '09'},
 {'id': 'DC',
  'loan_amnt': 15815.3939507095,
  'annual_inc': 94553.0708159074,
  'state': '11'},
 {'id': 'DE',
  'loan_amnt': 14935.1153607928,
  'annual_inc': 76689.2927407866,
  'state': '10'},
 {'id': 'FL',
  'loan_amnt': 14402.247655734,
  'annual_inc': 73171.6116762043,
  'state': '12'},
 {'id': 'GA',

In [12]:
loan_amnt_zip_new

[{'id': 'AK',
  'zip_code': '995xx',
  'loan_amnt': 17210.0100200401,
  'annual_inc': 80652.8329819639,
  'county': 'Anchorage',
  'state': '02'},
 {'id': 'AK',
  'zip_code': '996xx',
  'loan_amnt': 18103.0887230514,
  'annual_inc': 83295.4713018242,
  'county': 'Yukon-Koyukuk Census',
  'state': '02'},
 {'id': 'AK',
  'zip_code': '997xx',
  'loan_amnt': 17266.4808917197,
  'annual_inc': 77775.3338747346,
  'county': 'Fairbanks North Star',
  'state': '02'},
 {'id': 'AK',
  'zip_code': '998xx',
  'loan_amnt': 16078.2035175879,
  'annual_inc': 72974.7292713568,
  'county': 'Juneau City and',
  'state': '02'},
 {'id': 'AK',
  'zip_code': '999xx',
  'loan_amnt': 16130.4276315789,
  'annual_inc': 68294.7973026316,
  'county': 'Ketchikan Gateway',
  'state': '02'},
 {'id': 'AL',
  'zip_code': '350xx',
  'loan_amnt': 14847.8286189684,
  'annual_inc': 69053.9965890183,
  'county': 'St. Clair',
  'state': '01'},
 {'id': 'AL',
  'zip_code': '351xx',
  'loan_amnt': 15151.7017465293,
  'annual_in

In [13]:
a = zipcodes.similar_to('156')

In [14]:
res = [ sub['county'].rsplit(' ', 1)[0] for sub in a ] 

In [15]:
list(set(res))  

['Fayette', 'Westmoreland', 'Indiana', 'Armstrong']